In [1]:
from census import Census
import us
from us import states
import os
import pandas as pd
import geopandas as gpd
import requests
import re

api_key = os.environ["CENSUS_API_KEY"]

In [2]:
def nice_name(var):
    var = var.replace("Estimate!!", "")
    var = var.lower()

    var = re.sub(r'^[^a-zA-Z0-9]+|[^a-zA-Z0-9]+$', '',
                 re.sub(r'[^a-zA-Z0-9]+', '_', var))

    if var.startswith("total_"):
        var = var[6:]

    var = "_".join(var.split(" "))
    if var.endswith(":"):
        var = var[:-1]
    return var


def col_name(var):
    if "!!" in var:
        var = var.split("!!")[-1]
    return nice_name(var)

In [3]:
table = "B25003"
url = f"https://api.census.gov/data/2023/acs/acs5/groups/{table}.json"
r = requests.get(url)
data = r.json()
data = data["variables"]
vars = dict([(k, col_name(v["label"])) for k, v in data.items() if not k.startswith(table) or k.endswith("E")])
vars


{'NAME': 'geographic_area_name',
 'B25003_003E': 'renter_occupied',
 'B25003_002E': 'owner_occupied',
 'B25003_001E': 'total',
 'GEO_ID': 'geography'}

In [4]:
# build the Census object
c = Census(api_key, year=2023)
# send the query: county* means all counties in the state
data = c.acs5.get(list(vars.keys()), {'for': 'county:*', 'in': f'state:{states.NY.fips}'})

# # data is a list of dictionaries, so we can convert it to a DataFrame
df = pd.DataFrame(data)

df.rename(columns=vars, inplace=True)


In [5]:

df["my_total"] = df.renter_occupied + df.owner_occupied
df["mismatch"] = df.total != df.my_total
df[df.mismatch == True]

,geographic_area_name,renter_occupied,owner_occupied,total,geography,state,county,my_total,mismatch
